In [1]:
import numpy as np
import os
import cv2
import tensorflow as tf
import pathlib
from PIL import Image
from numpy import asarray
#import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, metrics
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Activation, MaxPool2D, BatchNormalization, Dropout
import matplotlib.pyplot as plt
import datetime
from sklearn.model_selection import KFold, StratifiedKFold
import sklearn.metrics as metrics
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [2]:
folder_path = '/home/jaliyah/data'
biofilm_full_path = os.path.join(folder_path, 'biofilm')
nonbio_full_path = os.path.join(folder_path, 'single_cell')
my_bio_files_list = []
my_non_files_list = []

In [3]:
for file in os.listdir(biofilm_full_path):
    my_bio_files_list.append(os.path.join(biofilm_full_path,file))

In [4]:
img_width = 300
img_height = 300

In [5]:
total_img_bio = np.zeros((276,img_width,img_height,3), dtype=np.int16)
biofilm_label= np.ones(276)
print(total_img_bio.shape)
for (i,file) in enumerate(my_bio_files_list):
    img = cv2.imread(file)
   # print(file)
   # plt.imshow(img)
   # plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    img_1 = img.resize(img_width,img_height,3)
    total_img_bio[i] = img
   # print(biofilm_label[i])

(276, 300, 300, 3)


In [6]:
for file in os.listdir(nonbio_full_path):
    my_non_files_list.append(os.path.join(nonbio_full_path,file))
#print((my_non_files_list[63]))

In [7]:
total_img_non = np.zeros((190,img_width,img_height,3), dtype=np.int16)
print(total_img_non.shape)
non_biofilm_label= np.zeros(190)

for (i,file) in enumerate(my_non_files_list):
    img = cv2.imread(file)
   # print(os.path.exists(file))
    #print(file)
    img_1 = img.resize(img_width,img_height,3)
    #print(file)
    #print (img.shape)
    #print(img)
    total_img_non[i] = img

(190, 300, 300, 3)


In [8]:
import random
total = total_img_bio.shape[0]
size = int(total * .20)
test_bio = random.sample(range(0, total), size)
np.unique(test_bio).shape

(55,)

In [9]:
test_bio_img = total_img_bio[test_bio]
test_bio_lbl = biofilm_label[test_bio]
test_bio_img[0].shape

(300, 300, 3)

In [10]:
total = total_img_non.shape[0]
size = int(total * .20)
test_non = random.sample(range(0, total), size)
np.unique(test_non).shape

(38,)

In [11]:
test_non_img = total_img_non[test_non]
test_non_lbl = non_biofilm_label[test_non]

In [12]:
total_test_img = np.concatenate([test_bio_img,test_non_img])
total_test_lbl = np.concatenate([test_bio_lbl,test_non_lbl])

#np.save('Dataset/test_img.npy', total_test_img) 
#np.save('Dataset/test_lbl.npy', total_test_lbl)

In [13]:
a = list(range(total_img_bio.shape[0]))
#gives us total 2225 to use for train/vali
train_vali_total = (set(a) -(set(test_bio)))
len_train_vali_total = (len(train_vali_total))
print(len_train_vali_total)
size = int(len_train_vali_total * .10)
#size
vail_bio = random.sample(range(0, len_train_vali_total), size)
np.unique(vail_bio).shape
vail_bio_img = total_img_bio[vail_bio]
vail_bio_lbl = biofilm_label[vail_bio]

221


In [14]:
a = list(range(len_train_vali_total))
train_total = (set(a) -(set(vail_bio)))
train_bio = list(train_total)
train_bio_img = total_img_bio[train_bio]
train_bio_lbl = biofilm_label[train_bio]
train_bio_img.shape
x = np.array(train_bio)
print(np.unique(x).shape)

(199,)


In [15]:
a = list(range(total_img_non.shape[0]))
#gives us total 4897 to use for train/vali
train_vali_non_total = (set(a) -(set(test_non)))
len_train_vali_non_total = (len(train_vali_non_total))
print(len_train_vali_non_total)
size = int(len_train_vali_non_total * .10)
print(size)
vail_non = random.sample(range(0, len_train_vali_non_total), size)
vail_non_img = total_img_non[vail_non]
vail_non_lbl = non_biofilm_label[vail_non]

152
15


In [16]:
a = list(range(len_train_vali_non_total))
train_non_total = (set(a) -(set(vail_non)))
train_non = list(train_non_total)
train_non_img = total_img_non[train_non]
train_non_lbl = non_biofilm_label[train_non]

In [17]:
total_vali_img = np.concatenate([vail_bio_img,vail_non_img])
total_vali_lbl = np.concatenate([vail_bio_lbl,vail_non_lbl])
#np.save('Dataset/vali_img.npy', total_vali_img)
#np.save('Dataset/vali_lbl.npy', total_vali_lbl)

In [18]:
total_train_img = np.concatenate([train_bio_img,train_non_img])
total_train_lbl = np.concatenate([train_bio_lbl,train_non_lbl])
#np.save('Dataset/train_img.npy', total_train_img)
#np.save('Dataset/train_lbl.npy', total_train_lbl)

In [19]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(
        rotation_range=20,
        zoom_range=0.15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        horizontal_flip=True,
        fill_mode="nearest")

test_datagen = keras.preprocessing.image.ImageDataGenerator()

In [20]:
def build_model(input_shape):
    inputs = Input(shape = input_shape)
    #1st block
    x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
    x = Dropout(0.3)(x)
    #2nd block
    x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
    x = Dropout(0.3)(x)
    #3rd block
    x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
    x = Dropout(0.3)(x)
    #4th block
    """
    x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
    x = Dropout(0.3)(x)
    """
    # 5th Conv block

    x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
    x = Dropout(0.3)(x)
    # Fully connected layers  
    x = Flatten()(x) 
    x = Dense(units = 1024, activation ='relu', kernel_regularizer = 'l2')(x) 
    x = Dropout(0.3)(x)
    x = Dense(units = 1024, activation ='relu')(x) 
    x = Dropout(0.3)(x)
    outputs = Dense(units = 1, activation ='sigmoid')(x)
    return keras.Model(inputs,outputs)

In [21]:
def get_model_name(k):
    return 'model_'+str(k)+'.h5'

In [22]:
X =  np.concatenate([total_train_img,total_vali_img])
y = np.concatenate([total_train_lbl,total_vali_lbl])

VALIDATION_ACCURACY = []
VALIDATION_LOSS = []

save_dir = 'saved_models/'
fold_var = 1

kfold = KFold(n_splits=5, shuffle=True) 
# enumerate splits
for train_index, valid_index in kfold.split(X, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_valid = X[train_index], X[valid_index]
    y_train, y_valid = y[train_index], y[valid_index]

    train_generator = train_datagen.flow(X_train, y_train, batch_size=32, shuffle=True)
    valid_generator = train_datagen.flow(X_valid, y_valid, batch_size=32, shuffle=True)
    model = build_model((img_height,img_width,3))
    from keras import backend as K

    def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    def f1_m(y_true, y_pred):
        precision = precision_m(y_true, y_pred)
        recall = recall_m(y_true, y_pred)
        return 2*((precision*recall)/(precision+recall+K.epsilon()))

    model.compile(loss='binary_crossentropy', 
           optimizer=tf.optimizers.Adam(learning_rate=0.00001),  
           metrics=['accuracy',f1_m,precision_m, recall_m])
    logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))  
    my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5),
    tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1),
    tf.keras.callbacks.ModelCheckpoint(save_dir+get_model_name(fold_var), 
                                        monitor='val_accuracy', verbose=1, 
                                        save_best_only=True, mode='max')
    ]

    model.fit(train_generator, validation_data=valid_generator,
                    callbacks=my_callbacks, epochs=50)
    model.load_weights("saved_models/model_"+str(fold_var)+".h5")

    results = model.evaluate(valid_generator)
    results = dict(zip(model.metrics_names,results))

    VALIDATION_ACCURACY.append(results['accuracy'])
    VALIDATION_LOSS.append(results['loss'])

    tf.keras.backend.clear_session()

    fold_var += 1
   # loss, accuracy,Precision, Recall = model.evaluate(valid_generator)

2022-08-01 17:28:18.860228: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-01 17:28:18.895283: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-01 17:28:18.895522: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-01 17:28:18.896702: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Epoch 1/50


2022-08-01 17:28:22.545139: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8300
2022-08-01 17:28:23.561152: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022-08-01 17:28:23.563186: W tensorflow/stream_executor/gpu/asm_compiler.cc:230] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.6
2022-08-01 17:28:23.563220: W tensorflow/stream_executor/gpu/asm_compiler.cc:233] Used ptxas at ptxas
2022-08-01 17:28:23.563344: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be 

10/10 [==============================] - ETA: 0s - loss: 20.9957 - accuracy: 0.7919 - f1_m: 0.8350 - precision_m: 0.8430 - recall_m: 0.8626

2022-08-01 17:28:32.714044: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 45424312320 exceeds 10% of free system memory.


...


Epoch 00004: val_accuracy did not improve from 0.82667
10/10 [==============================] - 13s 1s/step - loss: 20.4080 - accuracy: 0.9430 - f1_m: 0.9525 - precision_m: 0.9637 - recall_m: 0.9444 - val_loss: 21.1693 - val_accuracy: 0.4533 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 5/50
10/10 [==============================] - ETA: 0s - loss: 20.3148 - accuracy: 0.9362 - f1_m: 0.9443 - precision_m: 0.9345 - recall_m: 0.9578

2022-08-01 17:29:43.515256: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 45424312320 exceeds 10% of free system memory.



Epoch 00005: val_accuracy did not improve from 0.82667
10/10 [==============================] - 13s 1s/step - loss: 20.3148 - accuracy: 0.9362 - f1_m: 0.9443 - precision_m: 0.9345 - recall_m: 0.9578 - val_loss: 21.6645 - val_accuracy: 0.4533 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 6/50
10/10 [==============================] - ETA: 0s - loss: 20.1877 - accuracy: 0.9497 - f1_m: 0.9522 - precision_m: 0.9590 - recall_m: 0.9512
Epoch 00006: val_accuracy did not improve from 0.82667
10/10 [==============================] - 13s 1s/step - loss: 20.1877 - accuracy: 0.9497 - f1_m: 0.9522 - precision_m: 0.9590 - recall_m: 0.9512 - val_loss: 22.1484 - val_accuracy: 0.4533 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 7/50
10/10 [==============================] - ETA: 0s - loss: 20.0363 - accuracy: 0.9799 - f1_m: 0.9844 - precision_m: 0.9806 - recall_m: 0.9892
Epoch 00007: val_accuracy did not improve from 0.8266

Epoch 6/50
10/10 [==============================] - ETA: 0s - loss: 20.3052 - accuracy: 0.9396 - f1_m: 0.9419 - precision_m: 0.9243 - recall_m: 0.9659
Epoch 00006: val_accuracy did not improve from 0.76000
10/10 [==============================] - 13s 1s/step - loss: 20.3052 - accuracy: 0.9396 - f1_m: 0.9419 - precision_m: 0.9243 - recall_m: 0.9659 - val_loss: 22.9534 - val_accuracy: 0.4133 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 7/50
10/10 [==============================] - ETA: 0s - loss: 20.2008 - accuracy: 0.9396 - f1_m: 0.9538 - precision_m: 0.9537 - recall_m: 0.9580
Epoch 00007: val_accuracy did not improve from 0.76000
10/10 [==============================] - 13s 1s/step - loss: 20.2008 - accuracy: 0.9396 - f1_m: 0.9538 - precision_m: 0.9537 - recall_m: 0.9580 - val_loss: 23.6092 - val_accuracy: 0.4133 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 8/50
10/10 [==============================] - ET

Epoch 8/50
10/10 [==============================] - ETA: 0s - loss: 20.1778 - accuracy: 0.9264 - f1_m: 0.9331 - precision_m: 0.9214 - recall_m: 0.9530
Epoch 00008: val_accuracy did not improve from 0.60811
3/3 [==============================] - 1s 206ms/step - loss: 21.7410 - accuracy: 0.6081 - f1_m: 0.7529 - precision_m: 0.6062 - recall_m: 1.0000


In [23]:
test_generator = test_datagen.flow(total_test_img,total_test_lbl)
loss, accuracy,Precision, Recall = model.evaluate(test_generator)

3/3 [==============================] - 1s 474ms/step - loss: 21.8281 - accuracy: 0.5914 - f1_m: 0.7435 - precision_m: 0.5934 - recall_m: 1.0000


ValueError: too many values to unpack (expected 4)

In [ ]:
VALIDATION_ACCURACY

In [ ]:
np.mean(VALIDATION_ACCURACY)

In [ ]:
# summarize feature map shapes
for i in range(len(model.layers)):
    layer = model.layers[i]
    # check for convolutional layer
    if 'conv' not in layer.name:
        continue
    # summarize output shape
    print(i, layer.name, layer.output.shape)


In [ ]:
ixs = [3, 7, 9, 13, 15]
outputs = [model.layers[i+1].output for i in ixs]
model1 = tf.keras.Model(inputs=model.inputs, outputs=outputs)
model1.summary()

In [ ]:
test_dir = pathlib.Path('/home/jaliyah/data/single_cell//')
img_path = os.path.join(test_dir, 'slide eleven (464).jpg')

test_img = keras.preprocessing.image.load_img(img_path, target_size=(img_height, img_width))
img_array = keras.preprocessing.image.img_to_array(test_img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis
print(img_array.shape)


In [ ]:
len(feature_maps)

In [ ]:
import matplotlib.pyplot as pyplot
square = 8

f = plt.figure(figsize=(15,15))
ix = 1
for _ in range(square):
    for _ in range(square):
        # specify subplot and turn of axis
        ax = pyplot.subplot(square, square, ix)
        ax.set_xticks([])
        ax.set_yticks([])
        # plot filter channel in grayscale
        pyplot.imshow(np.squeeze(feature_maps[1])[:, :, ix-1])
        ix += 1
# show the figure
pyplot.show()

In [ ]:
filters, biases = model.layers[9].get_weights()


In [ ]:
f_min, f_max = filters.min(), filters.max()
filters = (filters - f_min) / (f_max - f_min)

In [ ]:
n_filters, ix = 6, 1
for i in range(n_filters):
    # get the filter
    f = filters[:, :, :, i]
    # plot each channel separately
    for j in range(3):
        # specify subplot and turn of axis
        ax = pyplot.subplot(n_filters, 3, ix)
        ax.set_xticks([])
        ax.set_yticks([])
        # plot filter channel in grayscale
        pyplot.imshow(f[:, :, j], cmap='gray')
        ix += 1
# show the figure
pyplot.show()

In [ ]:
test_dir = pathlib.Path('/home/jaliyah/data/single_cell//')
img_path = os.path.join(test_dir, 'slide eleven (313).jpg')

test_img = keras.preprocessing.image.load_img(img_path, target_size=(img_height, img_width))
img_array = keras.preprocessing.image.img_to_array(test_img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis
print(img_array.shape)

In [ ]:
def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:
img_array.shape

In [ ]:
last_conv_layer_name = 'conv2d_4'
heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)


In [ ]:
plt.matshow(heatmap)
plt.show()

In [ ]:
model.predict(img_array)

In [ ]:
def save_and_display_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=0.4):
    # Load the original image
    img = keras.preprocessing.image.load_img(img_path)
    img = keras.preprocessing.image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM1
    display(Image(cam_path))



In [ ]:
img_path

In [ ]:
save_and_display_gradcam(img_path, heatmap)